In [1]:
%pip install transformers==4.32 \
             datasets -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kaggle-environments 1.14.15 requires transformers>=4.33.1, but you have transformers 4.32.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import csv
import pandas as pd
from datasets import Dataset

# **Dataset preparation:**

In [3]:
h_path= '/kaggle/input/test-dataset/test sentences hindi.txt'

In [4]:
def read_and_filter(file_path):
    sentences= []
    with open(file_path, 'r', encoding= 'utf-8') as file:
        for line in file:
            stripped_line= line.strip()
            if stripped_line:
                sentences.append(line)
                
    return sentences

h_sen= read_and_filter(h_path)

In [5]:
print(f"Total number of sentences: {len(h_sen)}")

Total number of sentences: 500


In [6]:
dataset= list(h_sen)

file_name= "hi_test.csv"

with open(file_name, 'w', encoding= 'utf-8') as file:
    writer= csv.writer(file)
    writer.writerow(['Hindi'])
    
    for Hindi in dataset:
        writer.writerow([Hindi])

In [8]:
dataframe= pd.read_csv("/kaggle/working/hi_test.csv")
dataframe

,Hindi
0,हिन्दी-हिमाचली पहाड़ी भाषा का नाम सुनकर ही कुछ...
1,अगर इस मुद्दे पर दूसरे प्रदेशों के निवासियों क...
2,किंतु अपने ही प्रदेश की भाषा के प्रति लोगों की...
3,पता नहीं ऐसा कर के क्या हासिल‌ होगा ।\n
4,अगर हम अपनी माता ( हिमाचली भाषा ) के नहीं हो स...
...,...
495,"मुझे दो मरने, फिर देखना, कितने बीस सौ होता है।\n"
496,सब्जी तो गैस पर अच्छी बन जाती है लेकिन चपातिया...
497,"लीना चूल्हे की नहीं खाती, कहती है, राख लगी होत..."
498,"क्या करें चार चूल्हे में पकानी पड़ती हैं, बाकी..."


In [9]:
test_dataset= Dataset.from_pandas(dataframe)

# **Preparing the tokenizer:**

In [11]:
def fix_tokenizer(tokenizer, new_lang='kan_Deva'):

    old_len = len(tokenizer) - int(new_lang in tokenizer.added_tokens_encoder)
    tokenizer.lang_code_to_id[new_lang] = old_len-1
    tokenizer.id_to_lang_code[old_len-1] = new_lang

    tokenizer.fairseq_tokens_to_ids["<mask>"] = len(tokenizer.sp_model) + len(tokenizer.lang_code_to_id) + tokenizer.fairseq_offset

    tokenizer.fairseq_tokens_to_ids.update(tokenizer.lang_code_to_id)
    tokenizer.fairseq_ids_to_tokens = {v: k for k, v in tokenizer.fairseq_tokens_to_ids.items()}
    if new_lang not in tokenizer._additional_special_tokens:
        tokenizer._additional_special_tokens.append(new_lang)
    tokenizer.added_tokens_encoder = {}
    tokenizer.added_tokens_decoder = {}
    
    return tokenizer

> **1. Kinnauri**

In [12]:
from transformers import NllbTokenizer
s_lang="hin_Deva"
t_lang="kan_Deva"
k_tokenizer = NllbTokenizer.from_pretrained('facebook/nllb-200-distilled-600M', src_lang=s_lang, tgt_lang=t_lang)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

In [13]:
k_tokenizer= fix_tokenizer(k_tokenizer, new_lang='kan_Deva')

> **2. Kangri**

In [14]:
s_lang="hin_Deva"
t_lang="kangri_Deva"
kan_tokenizer = NllbTokenizer.from_pretrained('facebook/nllb-200-distilled-600M', src_lang=s_lang, tgt_lang=t_lang)

In [15]:
kan_tokenizer= fix_tokenizer(kan_tokenizer, new_lang='kangri_Deva')

# **Generating translations:**

In [21]:
from transformers import AutoModelForSeq2SeqLM, pipeline

2024-08-06 06:13:06.971128: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-06 06:13:06.971229: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-06 06:13:07.140575: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


> **1. Kinnauri**

In [16]:
kin_model= '/kaggle/input/nllb_ft_kinnauri/pytorch/default/1'
source= 'hin_Deva'
target= 'kan_Deva'

In [19]:
kin_model= AutoModelForSeq2SeqLM.from_pretrained(kin_model)

In [22]:
kintranslator = pipeline('translation', model=kin_model, tokenizer=k_tokenizer, src_lang=source, tgt_lang=target)

In [24]:
df = pd.DataFrame(test_dataset)

translations = []
for index, row in df.iterrows():
    hindi_text = row['Hindi']

    translation = kintranslator(hindi_text, max_length=128)[0]['translation_text']

    translations.append({
        'Hindi': hindi_text,
        'Generated Kinori': translation
    })

results_df = pd.DataFrame(translations)
results_df

,Hindi,Generated Kinori
0,हिन्दी-हिमाचली पहाड़ी भाषा का नाम सुनकर ही कुछ...,हिन्दी-हिमाचली पहाड़ी भाषा रोह नाऔ शूनिंदैए दौ...
1,अगर इस मुद्दे पर दूसरे प्रदेशों के निवासियों क...,कियै हौयौ मुसौल कै ओरिकै प्रदेशौ रै निवासी लै ...
2,किंतु अपने ही प्रदेश की भाषा के प्रति लोगों की...,तेबिह आफरोऐ प्रदेश रोह भाषा रोह ताई मांनुशा रो...
3,पता नहीं ऐसा कर के क्या हासिल‌ होगा ।\n,ज़ानी हैऔ कौरिंदै कयौ हासिल फिरेस।
4,अगर हम अपनी माता ( हिमाचली भाषा ) के नहीं हो स...,कियै तामोरि आपनी माता (हिमाचली भाषा) रिह निफिर...
...,...,...
495,"मुझे दो मरने, फिर देखना, कितने बीस सौ होता है।\n","मुला दुई मौरना, तेका शाँबै, केति बिश सौ थाकदौस।"
496,सब्जी तो गैस पर अच्छी बन जाती है लेकिन चपातिया...,सब्जी ता ग्यास कै भौली बौनिंदिस पर चपाटी ता बा...
497,"लीना चूल्हे की नहीं खाती, कहती है, राख लगी होत...","लीना च़िलै रि नाईं खाँदी, बोलदि, खोर लागी थाकदिस।"
498,"क्या करें चार चूल्हे में पकानी पड़ती हैं, बाकी...","कयौ कौरना च़ार चूल्हे कै पकाँबै च़ानदैसा, बाकी..."


In [36]:
pd.set_option('display.max_rows', None)  
pd.set_option('display.max_columns', None)  
pd.set_option('display.max_colwidth', None)  
pd.set_option('display.max_seq_item', None)  

In [37]:
results_df

,Hindi,Generated Kinori
0,हिन्दी-हिमाचली पहाड़ी भाषा का नाम सुनकर ही कुछ लोग विदक जाते हैं।\n,हिन्दी-हिमाचली पहाड़ी भाषा रोह नाऔ शूनिंदैए दौक मानुष विदक नाशदैसा।
1,अगर इस मुद्दे पर दूसरे प्रदेशों के निवासियों को कोई तकलीफ हो तो भी समझ आती है ।\n,कियै हौयौ मुसौल कै ओरिकै प्रदेशौ रै निवासी लै किछ़ तकलीफ हुतांह बिह सौमज़ आछ़दौस।
2,किंतु अपने ही प्रदेश की भाषा के प्रति लोगों की के मन की संकीर्णता समझ नहीं आती ।\n,तेबिह आफरोऐ प्रदेश रोह भाषा रोह ताई मांनुशा रोह मोन रोह संकोराई ज़ानोह नाछ़तोस।
3,पता नहीं ऐसा कर के क्या हासिल‌ होगा ।\n,ज़ानी हैऔ कौरिंदै कयौ हासिल फिरेस।
4,अगर हम अपनी माता ( हिमाचली भाषा ) के नहीं हो सकते तो विमाता के कहाँ होगे ।\n,कियै तामोरि आपनी माता (हिमाचली भाषा) रिह निफिरी सौकदी ता विमाता रै कियै फिरेस।
5,ऐसा तो है नहीं कि हिमाचली पहाड़ी का पुराना अस्तित्व या कोई इतिहास है ही नहीं और ये नई - नई बनाई गई है ।\n,होयो ताह साह नुवाँ किह हिमाचली पहाडी रोह पुरानो अस्तित्व याह किछ़े इतिहास साह नाई आ होई नोईयोह-नोईयोह ज़ुर्राई साह।
6,हिमाचली पहाड़ी भाषा के साथ यहां के स्थानीय लोगों का प्यार सदियों पुराना है।\n,हिमाचली पहाड़ी भाषा सिह ईन्छ़े रोह स्थानीय मांनुशा रोह बेदान सेग पुरानोस।
7,हालांकि सन 920 ई से पहले का साहित्य लिखित रूप में नहीं मिलता । \n,हालांकि सन 920 ई का पैलै रौ साहित्य लिखित रुह कै नौपौरदौस।
8,जो कुछ मिलता भी है वो लोकगीतों और भजनों के रूप में पीढ़ी दर पीढ़ी चलता आ रहा है ।\n,ज़ौ किछ़ पौरदौ बि हौसौ लोकगीतौ आ भजनौ रोह रुवा कै पीढ़ी-पीढ़ी च़ौलान आछ़ानस।
9,"शिलालेख / राजाओं के लिखे पत्र /संधिपत्र , चर्पट नाथ लिखित चरपट मंजरी इसके बाद के शुुरुआती लिखित सबूत हैं।\n","शिलालेख /राज़ा होर रोह चेमाईंदौ च़िठिपत्र, चर्पट नाथ चेमाईंदौ च़िठि मन्ज़ी हेथा पिछ़ु रोह शुरुका लिखित सबूतस।"


> **2. Kangri**

In [26]:
kan_model= '/kaggle/input/nllb_ft_kangri/pytorch/default/1'
source= 'hin_Deva'
target= 'kangri_Deva'

In [27]:
kan_model= AutoModelForSeq2SeqLM.from_pretrained(kan_model)

In [28]:
kantranslator = pipeline('translation', model=kan_model, tokenizer=kan_tokenizer, src_lang=source, tgt_lang=target)

In [38]:
df = pd.DataFrame(test_dataset)

translations = []
for index, row in df.iterrows():
    hindi_text = row['Hindi']

    translation = kantranslator(hindi_text, max_length=128)[0]['translation_text']

    translations.append({
        'Hindi': hindi_text,
        'Generated kangri': translation
    })

results_df_kangri = pd.DataFrame(translations)
results_df_kangri

,Hindi,Generated kangri
0,हिन्दी-हिमाचली पहाड़ी भाषा का नाम सुनकर ही कुछ लोग विदक जाते हैं।\n,हिन्दी-हिमाचली पहाड़ी भाषा दा नां सुणी करी ही कुछ लोक विदक जांदे हन।
1,अगर इस मुद्दे पर दूसरे प्रदेशों के निवासियों को कोई तकलीफ हो तो भी समझ आती है ।\n,जे इस मुसले पर दूएं प्रदेशां दे निवासीयां जो कोई दिक्खी हो तां बी समझ औंदी
2,किंतु अपने ही प्रदेश की भाषा के प्रति लोगों की के मन की संकीर्णता समझ नहीं आती ।\n,पर अपणे ही प्रदेश दी भाषा दी तरफ़ लोकां दे मने दी संकीर्णता समझी नी औंदी
3,पता नहीं ऐसा कर के क्या हासिल‌ होगा ।\n,पता नी ऐसा करी के क्या हासिल हुंगा
4,अगर हम अपनी माता ( हिमाचली भाषा ) के नहीं हो सकते तो विमाता के कहाँ होगे ।\n,जे असां अपणी माता (महाचली भाषा) दे नी हो सकदे तां विमाता दे कुत्थु हुंगे
5,ऐसा तो है नहीं कि हिमाचली पहाड़ी का पुराना अस्तित्व या कोई इतिहास है ही नहीं और ये नई - नई बनाई गई है ।\n,एह ता है नी कि म्हाचली पहाड़ी दा पुराना अस्तित्व या कोई इतिहास है ही नी कनै ये नयी नयी बणाई गई
6,हिमाचली पहाड़ी भाषा के साथ यहां के स्थानीय लोगों का प्यार सदियों पुराना है।\n,हिमाचल पहाड़ी भाषा कने इत्थू दे स्थानीय लोकां दा प्यार सदियां पुराना ए।
7,हालांकि सन 920 ई से पहले का साहित्य लिखित रूप में नहीं मिलता । \n,हालांकि सन 920 ई ते पैले दा साहित्य लिखित रूपे च नी मिलदा।
8,जो कुछ मिलता भी है वो लोकगीतों और भजनों के रूप में पीढ़ी दर पीढ़ी चलता आ रहा है ।\n,जे कुछ मिलदा बी सैह लोकगीतां कने भजनों दे रूपे च पीढ़ी दर पीढ़ी चलदा औआ दा
9,"शिलालेख / राजाओं के लिखे पत्र /संधिपत्र , चर्पट नाथ लिखित चरपट मंजरी इसके बाद के शुुरुआती लिखित सबूत हैं।\n","शिलालेख / राजेयां दे लिखे पत्र /संधिपत्र , चर्पट नाथ लिखित चरपट मंजरी इसते बाद दे सुरुआती लिखित सबूत हन।"
